In [103]:
import dataiku
import dataikuapi
from dataiku import pandasutils as pdu
import pandas as pd
import logging

In [104]:
design_host = dataiku.api_client().host 
print(design_host)

http://127.0.0.1:20001


In [105]:
design_client = dataikuapi.DSSClient(design_host, "FJk7Jffd8TStmT4Zvx9efC6G5mYZusec") 

In [106]:
curr_log_user = design_client.get_own_user().get_settings().get_raw()["login"]

In [86]:
dss_users = design_client.list_users()
user_info = [user for user in dss_users if user['login'] == curr_log_user]
if 'MLOps' in user_info[0]['groups']:
    logging.info("User {} is in the 'MLOps' group".format(curr_log_user))
else:
    logging.error("User {} is not in the 'MLOps' group".format(curr_log_user))
    raise Exception("User {} is not in the 'MLOps' group".format(curr_log_user))
        

In [107]:
project = design_client.get_project("DSPIPELINEFRAUD")

In [108]:
project_permissions = project.get_permissions().get("permissions")

In [98]:
#bundle_version_type = config.get("bundle_version_type", "")
bundle_version_type = "minor"

In [99]:
#exported_bundles = ["DSPIPELINEFRAUD_v1", "DSPIPELINEFRAUD_v1.1", "DSPIPELINEFRAUD_v1.2", "DSPIPELINEFRAUD_v2", "DSPIPELINEFRAUD_v2.1"]

In [100]:
#version_id = 1
exported_bundles = project.list_exported_bundles()['bundles']
list_bundles = []
for bundles in exported_bundles:
    list_bundles.append(bundles['bundleId'])
if len(list_bundles) == 0: 
    if bundle_version_type == "major":
        version_id = 1
    elif bundle_version_type == "minor":
        raise Exception("This is the first time you deploy the project, it should be a major version!") 
elif len(list_bundles) > 0:
    last_deployed_bundles = list_bundles[-1]
    last_version = last_deployed_bundles.split('v')[1].split('_')
    if bundle_version_type == "major":
        version_id = int(last_version[0]) + 1
    elif bundle_version_type == "minor":
        if len(last_version) == 1:
            version_id = last_version[0] + '_1'
        elif len(last_version) == 2:
            version_id = last_version[0] + '_' + str(int(last_version[1]) + 1)

bundle_id = project_key + '_v' + str(version_id)

In [101]:
bundle_id

'DSPIPELINEFRAUD_v1_4'

In [92]:
list_bundles = list(set(list_bundles))

In [93]:
list_bundles

['DSPIPELINEFRAUD_v1_3',
 'DSPIPELINEFRAUD_v1_2',
 'DSPIPELINEFRAUD_v1',
 'DSPIPELINEFRAUD_v1_1']

'DSPIPELINEFRAUD_v1_1'

In [37]:
project.list_exported_bundles()

{'bundles': [{'bundleId': 'DSPIPELINEFRAUD_v2',
   'contentSummary': {'allAnalysisData': False,
    'configContent': {'analysis': 2,
     'dashboards': 3,
     'datasets': 24,
     'insights': 16,
     'jupyterNotebooks': 3,
     'managedFolders': 1,
     'modelEvaluationStores': 0,
     'recipes': 16,
     'savedModels': 1,
     'scenarios': 0,
     'sqlNotebooks': 0},
    'includedDatasets': [{'name': 'transactions_2018',
      'size': 5404878,
      'type': 'UploadedFiles',
      'usedTransferMethod': 'UPLOAD'},
     {'name': 'transactions_2017',
      'size': 7423251,
      'type': 'UploadedFiles',
      'usedTransferMethod': 'UPLOAD'},
     {'name': 'merchant_info',
      'size': 3362746,
      'type': 'UploadedFiles',
      'usedTransferMethod': 'UPLOAD'},
     {'name': 'cardholder_info',
      'size': 2543734,
      'type': 'UploadedFiles',
      'usedTransferMethod': 'UPLOAD'}],
    'includedManagedFolders': [],
    'includedModelEvaluationStores': [],
    'includedSavedModels'

In [38]:
host = "http://localhost:15000/"
auto_client = dataikuapi.DSSClient(host, "Da1cTKi0eO3VMUUvCq7TuIfSHRBzY3OA") 
project = auto_client.get_project("DSPIPELINEFRAUD")

In [39]:
project.list_imported_bundles()

{'bundles': [{'bundleId': 'DSPIPELINEFRAUD_v2',
   'contentSummary': {'allAnalysisData': False,
    'configContent': {'analysis': 2,
     'dashboards': 3,
     'datasets': 24,
     'insights': 16,
     'jupyterNotebooks': 3,
     'managedFolders': 1,
     'modelEvaluationStores': 0,
     'recipes': 16,
     'savedModels': 1,
     'scenarios': 0,
     'sqlNotebooks': 0},
    'includedDatasets': [{'name': 'transactions_2018',
      'size': 5404878,
      'type': 'UploadedFiles',
      'usedTransferMethod': 'UPLOAD'},
     {'name': 'transactions_2017',
      'size': 7423251,
      'type': 'UploadedFiles',
      'usedTransferMethod': 'UPLOAD'},
     {'name': 'merchant_info',
      'size': 3362746,
      'type': 'UploadedFiles',
      'usedTransferMethod': 'UPLOAD'},
     {'name': 'cardholder_info',
      'size': 2543734,
      'type': 'UploadedFiles',
      'usedTransferMethod': 'UPLOAD'}],
    'includedManagedFolders': [],
    'includedModelEvaluationStores': [],
    'includedSavedModels'

In [109]:
writers = [permission["group"] for permission in project_permissions if permission.get("writeProjectContent", False)==True]

In [110]:
writers

['MLOps', 'cc_fraud_project']

In [56]:
project_deployer_client = dataikuapi.DSSClient('http://localhost:20000/', "FJk7Jffd8TStmT4Zvx9efC6G5mYZusec") 

In [57]:
project_owner = project.get_permissions().get("owner")
published_project = project_deployer_client.get_project("DSPIPELINEFRAUD") 

In [59]:
published_project

In [118]:
published_project_settings = published_project.get_settings()
raw_permissions = published_project_settings.get_raw()

In [119]:
raw_permissions

{'bundleExporterSettings': {'exportOptions': {'exportAllDatasets': False,
   'exportAllInputDatasets': False,
   'exportAllInputManagedFolders': False,
   'exportAnalysisModels': False,
   'exportGitRepository': False,
   'exportInsightsData': False,
   'exportManagedFS': False,
   'exportManagedFolders': False,
   'exportModelEvaluationStores': False,
   'exportSavedModels': False,
   'exportUploads': False,
   'includedDatasetsData': [{'name': 'transactions_2018',
     'type': 'UploadedFiles'},
    {'name': 'transactions_2017', 'type': 'UploadedFiles'},
    {'name': 'merchant_info', 'type': 'UploadedFiles'},
    {'name': 'cardholder_info', 'type': 'UploadedFiles'}],
   'includedManagedFolders': [],
   'includedModelEvaluationStores': [],
   'includedSavedModels': [],
   'shareAllInputDatasets': False,
   'useManualPluginsInfo': False}},
 'exposedObjects': {'objects': []},
 'imgPattern': 4,
 'metrics': {'displayedState': {'columns': [], 'metrics': []},
  'engineConfig': {'basic': {},


In [113]:
logging.info("Project owner will again be {}".format(curr_log_user))

In [114]:
# Set the new project owner on the Deployer
raw_permissions['owner'] = curr_log_user

In [115]:
raw_permissions['permissions']

KeyError: 'permissions'

In [116]:
raw_permissions

{'bundleExporterSettings': {'exportOptions': {'exportAllDatasets': False,
   'exportAllInputDatasets': False,
   'exportAllInputManagedFolders': False,
   'exportAnalysisModels': False,
   'exportGitRepository': False,
   'exportInsightsData': False,
   'exportManagedFS': False,
   'exportManagedFolders': False,
   'exportModelEvaluationStores': False,
   'exportSavedModels': False,
   'exportUploads': False,
   'includedDatasetsData': [{'name': 'transactions_2018',
     'type': 'UploadedFiles'},
    {'name': 'transactions_2017', 'type': 'UploadedFiles'},
    {'name': 'merchant_info', 'type': 'UploadedFiles'},
    {'name': 'cardholder_info', 'type': 'UploadedFiles'}],
   'includedManagedFolders': [],
   'includedModelEvaluationStores': [],
   'includedSavedModels': [],
   'shareAllInputDatasets': False,
   'useManualPluginsInfo': False}},
 'exposedObjects': {'objects': []},
 'imgPattern': 4,
 'metrics': {'displayedState': {'columns': [], 'metrics': []},
  'engineConfig': {'basic': {},


In [117]:
for writer in writers:
    if writer == 'MLOps':
        permission = {
                'admin': True, 
                'deploy': True, 
                'group': writer, 
                'read': True, 
                'write': True
        }
        raw_permissions['permissions'].append(permission)

KeyError: 'permissions'

In [0]:
        
    # this part deduplicates any permissions given to the same group.
permissions = { each['group'] : each for each in raw_permissions['permissions'] }.values()
list_permissions = list(permissions)
raw_permissions['permissions'] = list_permissions
published_project_settings.save()